In [ ]:
# default_exp annotation.core

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

from enum import Enum

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Annotation
> Data annotation logic.

In [ ]:
# export


class AnnotationShape(Enum):
    """
    The supported annotation shape types
    """
    NONE = 'none'
    CIRCLE = 'circle'
    ELLIPSE = 'ellipse'
    POINT = 'point'
    POLYGON = 'polygon'
    RECTANGLE = 'rect'

    def __str__(self):
        return self.value

In [ ]:
# export


def parse_shape(shape_str):
    """
    Try to parse the shape from a string representation.
    `shape_str`: the shape as string
    return: the parsed AnnotationShape
    raises: `ValueError` if unsupported shape parsed
    """
    try:
        return AnnotationShape(shape_str)
    except ValueError:
        raise ValueError("Error, unsupported annotation shape: {}".format(shape_str))

In [ ]:
# export


class Annotation:
    """
    A annotation
    `shape`: the annotation shape
    `points_x`: a list of points x-coordinates
    `points_y`: a list of points y-coordinates
    `radius_x`: a radius on x-coordinate
    `radius_y`: a radius on y-coordinate
    `labels`: a set of annotation labels
    """

    def __init__(self, shape=AnnotationShape.NONE, points_x=None, points_y=None, radius_x=0, radius_y=0, labels=None):
        self.shape = shape
        self.points_x = [] if points_x is None else points_x
        self.points_y = [] if points_y is None else points_y
        self.radius_x = radius_x
        self.radius_y = radius_y
        self.labels = [] if labels is None else labels

In [ ]:
# export


class FileAnnotation:
    """
    The annotations for a file.
    `file_id`: a unique file identifier
    `file_name`: the file
    `file_size`: the file size
    `file_path`: the file path
    `annotations`: A list of annotations
    """

    def __init__(self, file_id=None, file_name=None, file_size=None, file_path=None, annotations=None):
        self.file_id = file_id
        self.file_name = file_name
        self.file_path = file_path
        self.file_size = file_size
        self.annotations: [Annotation] = [] if annotations is None else annotations

In [ ]:
# export


def convert_annotation(annotation: Annotation, target_shape: AnnotationShape):
    """
    Convert annotation shape and points to target shape.
    `annotation`: the annotation to convert
    `target_shape`: the target shape to convert to
    """
    if target_shape != annotation.shape:
        x_min = min(annotation.points_x) - annotation.radius_x if len(annotation.points_x) else 0
        x_max = max(annotation.points_x) + annotation.radius_x if len(annotation.points_x) else 0
        y_min = min(annotation.points_y) - annotation.radius_y if len(annotation.points_y) else 0
        y_max = max(annotation.points_y) + annotation.radius_y if len(annotation.points_y) else 0
        center_x = x_min + x_max - x_min
        center_y = y_min + y_max - y_min
        annotation.shape = target_shape
        if target_shape == AnnotationShape.NONE:
            annotation.points_x = []
            annotation.points_y = []
            annotation.radius_x = 0
            annotation.radius_y = 0
        elif target_shape == AnnotationShape.CIRCLE or target_shape == AnnotationShape.ELLIPSE:
            annotation.points_x = [center_x]
            annotation.points_y = [center_y]
            annotation.radius_x = x_max - center_x
            annotation.radius_y = y_max - center_y
        elif target_shape == AnnotationShape.POINT:
            annotation.points_x = [center_x]
            annotation.points_y = [center_y]
            annotation.radius_x = 0
            annotation.radius_y = 0
        elif target_shape == AnnotationShape.POLYGON:
            annotation.points_x = [x_min, x_min, x_max, x_max, x_min]
            annotation.points_y = [y_min, y_max, y_max, y_min, y_min]
            annotation.radius_x = 0
            annotation.radius_y = 0
        elif target_shape == AnnotationShape.RECTANGLE:
            annotation.points_x = [x_min, x_max]
            annotation.points_y = [y_min, y_max]
            annotation.radius_x = 0
            annotation.radius_y = 0
        else:
            raise NotImplementedError('unsupported conversion rect -> {}'.format(target_shape))

In [ ]:
# export


def annotation_bounding_box(annotation: Annotation):
    """
    Calculates the annotation bounding box.
    `annotation`: the annotation
    return: a tuple of points_x and points_y
    """
    bbox = deepcopy(annotation)
    convert_annotation(bbox, AnnotationShape.RECTANGLE)
    return bbox.points_x, bbox.points_y
